SQL-script для создания таблиц
```SQL
create table if not exists customer(
  customer_id int primary key,
  first_name text,
  last_name text,
  gender text,
  DOB date,
  job_title text,
  job_industry_category text,
  wealth_segment text,
  deceased_indicator text,
  owns_car text,
  address text,
  postcode int,
  state text,
  country text,
  property_valuation int
);

create table if not exists product(
  product_id int,
  brand text,
  product_line text,
  product_class text,
  product_size text,
  list_price decimal,
  standard_cost decimal
);

create table if not exists product_cor as
select * from 
(select *,row_number() over(partition by product_id order by list_price desc) as rn from product)
where rn = 1;
ALTER TABLE product_cor ADD PRIMARY KEY (product_id);

create table if not exists orders(
  order_id int primary key,
  customer_id int,
  order_date date,
  online_order bool,
  order_status text
);

create table if not exists order_items(
  order_item_id int primary key,
  order_id int,
  product_id int,
  quantity int,
  item_list_price_at_sale decimal,
  item_standard_cost_at_sale decimal
);

## Таблица customer
![alt text](img/1.png)

## Таблица product_cor
![alt text](img/2.png)

## Таблица orders
![alt text](img/3.png)

## Таблица order_items
![alt text](img/4.png)

Вывести распределение (количество) клиентов по сферам деятельности, отсортировав результат по убыванию количества.
```SQL
select job_industry_category, count(*) AS client_count
from customer
group by job_industry_category
order by client_count desc;



![alt text](img/5.png)

Найти общую сумму дохода (list_price*quantity) по всем подтвержденным заказам за каждый месяц по сферам деятельности клиентов. Отсортировать результат по году, месяцу и сфере деятельности.
```SQL
select date_part('year', o.order_date) as year, date_part('month', o.order_date) as month, c.job_industry_category, 
    sum(oi.item_list_price_at_sale  * oi.quantity) as total_amount
from orders o
join customer c on o.customer_id = c.customer_id
join order_items oi on o.order_id = oi.order_id
where o.order_status = 'Approved'
group by year, month, c.job_industry_category
order by year, month, c.job_industry_category;

![alt text](img/6.png)

Вывести количество уникальных онлайн-заказов для всех брендов в рамках подтвержденных заказов клиентов из сферы IT. Включить бренды, у которых нет онлайн-заказов от IT-клиентов, — для них должно быть указано количество 0.
```SQL
with it_unique_orders as (
    select o.order_id
    from orders o
    join customer c on o.customer_id = c.customer_id
    where o.online_order = true and o.order_status = 'Approved' and  c.job_industry_category = 'IT'
)
select pc.brand, count(distinct io.order_id) as unique_online_orders
from product_cor pc
left join order_items oi on pc.product_id = oi.product_id
left join it_unique_orders io on oi.order_id = io.order_id
group by pc.brand

![alt text](img/7.png)

Найти по всем клиентам: сумму всех заказов (общего дохода), максимум, минимум и количество заказов, а также среднюю сумму заказа по каждому клиенту. Отсортировать результат по убыванию суммы всех заказов и количества заказов. Выполнить двумя способами: используя только GROUP BY и используя только оконные функции. Сравнить результат.
```SQL
with order_metrics as (
    select o.customer_id, o.order_id, sum(oi.item_list_price_at_sale * oi.quantity) as total_sum
    from orders o
    join order_items oi on o.order_id = oi.order_id
    group by o.customer_id, o.order_id
)
select c.customer_id, sum(om.total_sum) as total_amount, max(om.total_sum) as max_order, 
    min(om.total_sum) as min_order, count(distinct om.order_id) as order_count,
    avg(om.total_sum) as avg_order
from customer c
join order_metrics om on c.customer_id = om.customer_id
group by c.customer_id
order by total_amount desc, order_count desc;

with order_metrics as (
    select o.customer_id, o.order_id, sum(oi.item_list_price_at_sale * oi.quantity) as total_sum
    from orders o
    join order_items oi on o.order_id = oi.order_id
    group by o.customer_id, o.order_id
)
select customer_id, sum(total_sum) over (partition by customer_id) as total_amount,
    max(total_sum) over (partition by customer_id) as max_order,
    min(total_sum) over (partition by customer_id) as min_order,
    count(*) over (partition by customer_id) as order_count,
    avg(total_sum) over (partition by customer_id) as avg_order
FROM order_metrics
ORDER BY total_amount DESC, order_count DESC;

Используя только GROUP BY
![alt text](img/8.png)
Используя только оконные функции
![alt text](img/13.png)

Найти имена и фамилии клиентов с топ-3 минимальной и топ-3 максимальной суммой транзакций за весь период (учесть клиентов, у которых нет заказов, приняв их сумму транзакций за 0).
```SQL
with customer_sum as (
    select c.customer_id, c.first_name, c.last_name,
        coalesce(sum(oi.item_list_price_at_sale * oi.quantity), 0) as total_amount
    from customer c
    left join orders o on c.customer_id = o.customer_id
    left join order_items oi on o.order_id = oi.order_id
    group by c.customer_id, c.first_name, c.last_name
),
top_customers as (
    select *, rank() over (order by total_amount) as rank_min,
        rank() over (order by total_amount desc) as rank_max
    from customer_sum
)
select first_name, last_name, total_amount
from top_customers
where rank_min <= 3 or rank_max <= 3
order by total_amount, first_name, last_name;

![alt text](img/9.png)

Вывести только вторые транзакции клиентов (если они есть) с помощью оконных функций. Если у клиента меньше двух транзакций, он не должен попасть в результат.
```SQL
select customer_id, order_id
from (
    select o.customer_id, o.order_id, row_number() over (partition by o.customer_id order by o.order_date) as rank
    from orders o
) ranked_orders
where rank = 2;

![alt text](img/10.png)

Вывести имена, фамилии и профессии клиентов, а также длительность максимального интервала (в днях) между двумя последовательными заказами. Исключить клиентов, у которых только один или меньше заказов.
```sql
with ordered_orders as (
    select o.customer_id, o.order_id, o.order_date,
        lead(o.order_date) over (partition by o.customer_id order by o.order_date) as next_order_date
    from orders o
),
diffs as (
    select customer_id, date_part('day', next_order_date - order_date::timestamp) as diff_days
    from ordered_orders
    where next_order_date is not null
),
max_diffs as (
    select customer_id, max(diff_days) as max_interval
    from diffs
    group by customer_id
)
select c.first_name, c.last_name, c.job_title, md.max_interval
from max_diffs md
join customer c on md.customer_id = c.customer_id
where md.max_interval is not null

![alt text](img/11.png)

Найти топ-5 клиентов (по общему доходу) в каждом сегменте благосостояния (wealth_segment). Вывести имя, фамилию, сегмент и общий доход. Если в сегменте менее 5 клиентов, вывести всех.
```sql
with customer_revenue as (
    select c.customer_id, c.first_name, c.last_name, c.wealth_segment,
        coalesce(sum(oi.item_list_price_at_sale * oi.quantity), 0) as total_amount
    from customer c
    left join orders o on c.customer_id = o.customer_id
    left join order_items oi on o.order_id = oi.order_id
    group by c.customer_id, c.first_name, c.last_name, c.wealth_segment
),
top_customers as (
    select *,
        rank() over (partition by wealth_segment order by total_amount desc) as rank
    from customer_revenue
)
select first_name, last_name, wealth_segment, total_amount
from top_customers
where rank <= 5
order by wealth_segment, total_amount desc;

![alt text](img/12.png)